In [13]:
# Import packages
import pandas as pd
import numpy as np
import re

# Set  number format
# pd.options.display.float_format = "{:,.2f}".format

# Global parameters and data import

Describe assumptions about parameters here

In [14]:
# Set parameters

# Set file path
MRIO_path = "../../data/exiobase/_pxp/IOT_2019_pxp/"
annex_file = "../data/annex.xlsx"

# Measure 1
impact_coeff_biogas = 3.5 / 155
NG_price = 25.66  # EUR/MWh
biogas_price = 69.78  # EUR/MWh see excel file
NG_LHV = 35.8  # High Heating Value
biogas_LHV = 21.5  # in (MJ/Nm3)

# Measure 2
impact_coeff_RES = 22.5 / 155
technical_coeff_CCGT = 0.44   # 44% energy efficiency
LCOE_gas = 82.61
LCOE_wind = 117.98
LCOE_solar = 101.10

In [15]:
# ---------------------------------------------------------
# IMPORT MATRICES
# ---------------------------------------------------------
# Z = pd.read_csv(f"{MRIO_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{MRIO_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{MRIO_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
x_exio = pd.read_csv(f"{MRIO_path}x.txt", delimiter="\t", header=0, index_col=[0, 1])

F = pd.read_csv(f"{MRIO_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0])
F_y = pd.read_csv(f"{MRIO_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0])
F_unit = pd.read_csv(f"{MRIO_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0])

# ---------------------------------------------------------
# GET LABEL
# ---------------------------------------------------------
region_and_sector_labels = A.index
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()


In [16]:
# # Use this to filter the relevant sectors
# MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
# sector_labels[MASK]

In [17]:
idx_EU27 = region_labels[:27]
idx_RU = ["RU"]

idx_NG = [
    "Natural gas and services related to natural gas extraction, excluding surveying",
    "Distribution services of gaseous fuels through mains",
    "Natural Gas Liquids",
    "Transportation services via pipelines",
]

idx_RU_NG = pd.MultiIndex.from_product(iterables=[idx_RU, idx_NG])
RU_NG_to_EU27 = A.loc[idx_RU_NG, idx_EU27]
RU_NG_to_EU27

region                                                            AT   
sector                                                    Paddy rice   
region sector                                                          
RU     Natural gas and services related to natural gas...          0  \
       Distribution services of gaseous fuels through ...          0   
       Natural Gas Liquids                                         0   
       Transportation services via pipelines                       0   

region                                                                    
sector                                                            Wheat   
region sector                                                             
RU     Natural gas and services related to natural gas...  0.000000e+00  \
       Distribution services of gaseous fuels through ...  0.000000e+00   
       Natural Gas Liquids                                 0.000000e+00   
       Transportation services via pipelines               5.429457e-08   

region                                                                        
sector                                                    Cereal grains nec   
region sector                                                                 
RU     Natural gas and services related to natural gas...      0.000000e+00  \
       Distribution services of gaseous fuels through ...      0.000000e+00   
       Natural Gas Liquids                                     0.000000e+00   
       Transportation services via pipelines                   3.347689e-08   

region                                                                              
sector                                                    Vegetables, fruit, nuts   
region sector                                                                       
RU     Natural gas and services related to natural gas...            0.000000e+00  \
       Distribution services of gaseous fuels through ...            0.000000e+00   
       Natural Gas Liquids                                           0.000000e+00   
       Transportation services via pipelines                         2.507524e-08   

region                                                                    
sector                                                        Oil seeds   
region sector                                                             
RU     Natural gas and services related to natural gas...  0.000000e+00  \
       Distribution services of gaseous fuels through ...  0.000000e+00   
       Natural Gas Liquids                                 0.000000e+00   
       Transportation services via pipelines               5.880551e-08   

region                                                                             
sector                                                    Sugar cane, sugar beet   
region sector                                                                      
RU     Natural gas and services related to natural gas...           0.000000e+00  \
       Distribution services of gaseous fuels through ...           0.000000e+00   
       Natural Gas Liquids                                          0.000000e+00   
       Transportation services via pipelines                        9.372485e-08   

region                                                                         
sector                                                    Plant-based fibers   
region sector                                                                  
RU     Natural gas and services related to natural gas...       0.000000e+00  \
       Distribution services of gaseous fuels through ...       0.000000e+00   
       Natural Gas Liquids                                      0.000000e+00   
       Transportation services via pipelines                    1.093948e-07   

region                                                                    
sector                                                        Crops 

# Measure 1: Biomethane scale up

In [18]:
idx_biogas = ["Biogas"]
idx_EU27_biogas = pd.MultiIndex.from_product(iterables=[idx_EU27, idx_biogas])
EU27_biogas = A.loc[idx_EU27_biogas, idx_EU27]
EU27_biogas

region                AT                                                   
sector        Paddy rice Wheat Cereal grains nec Vegetables, fruit, nuts   
region sector                                                              
AT     Biogas          0   0.0               0.0                     0.0  \
BE     Biogas          0   0.0               0.0                     0.0   
BG     Biogas          0   0.0               0.0                     0.0   
CY     Biogas          0   0.0               0.0                     0.0   
CZ     Biogas          0   0.0               0.0                     0.0   
DE     Biogas          0   0.0               0.0                     0.0   
DK     Biogas          0   0.0               0.0                     0.0   
EE     Biogas          0   0.0               0.0                     0.0   
ES     Biogas          0   0.0               0.0                     0.0   
FI     Biogas          0   0.0               0.0                     0.0   
FR     Biogas          0   0.0               0.0                     0.0   
GR     Biogas          0   0.0               0.0                     0.0   
HR     Biogas          0   0.0               0.0                     0.0   
HU     Biogas          0   0.0               0.0                     0.0   
IE     Biogas          0   0.0               0.0                     0.0   
IT     Biogas          0   0.0               0.0                     0.0   
LT     Biogas          0   0.0               0.0                     0.0   
LU     Biogas          0   0.0               0.0                     0.0   
LV     Biogas          0   0.0               0.0                     0.0   
MT     Biogas          0   0.0               0.0                     0.0   
NL     Biogas          0   0.0               0.0                     0.0   
PL     Biogas          0   0.0               0.0                     0.0   
PT     Biogas          0   0.0               0.0                     0.0   
RO     Biogas          0   0.0               0.0                     0.0   
SE     Biogas          0   0.0               0.0                     0.0   
SI     Biogas          0   0.0               0.0                     0.0   
SK     Biogas          0   0.0               0.0                     0.0   

region                                                                        
sector        Oil seeds Sugar cane, sugar beet Plant-based fibers Crops nec   
region sector                                                                 
AT     Biogas       0.0                    0.0                0.0       0.0  \
BE     Biogas       0.0                    0.0                0.0       0.0   
BG     Biogas       0.0                    0.0                0.0       0.0   
CY     Biogas       0.0                    0.0                0.0       0.0   
CZ     Biogas       0.0                    0.0                0.0       0.0   
DE     Biogas       0.0                    0.0                0.0       0.0   
DK     Biogas       0.0                    0.0                0.0       0.0   
EE     Biogas       0.0                    0.0                0.0       0.0   
ES     Biogas       0.0                    0.0                0.0       0.0   
FI     Biogas       0.0                    0.0                0.0       0.0   
FR     Biogas       0.0                    0.0                0.0       0.0   
GR     Biogas       0.0                    0.0                0.0       0.0   
HR     Biogas       0.0                    0.0                0.0       0.0   
HU     Biogas       0.0                    0.0                0.0       0.0   
IE     Biogas       0.0                    0.0                0.0       0.0   
IT     Biogas       0.0                    0.0                0.0       0.0   
LT     Biogas       0.0                    0.0                0.0       0.0   
LU     Biogas       0.0                    0.0                0.0       0.0   
LV     Biogas       0.0                    0.0              

In [19]:
# assumption 1: reduction x % in Energy = x % in monetary flow
# assumption 2: equal reduction for each sector/country
RU_NG_reduction_from_biogas = impact_coeff_biogas * RU_NG_to_EU27
new_RU_gas_to_EU27 = RU_NG_to_EU27 - RU_NG_reduction_from_biogas

# assumption 3: same substitution coeffs for each sector/country
price_coeff_biogas = NG_price / biogas_price
technical_coeff_biogas = NG_LHV / biogas_LHV
SWK_biogas = price_coeff_biogas * technical_coeff_biogas

increase_in_biogas = RU_NG_reduction_from_biogas / SWK_biogas
increase_in_biogas = increase_in_biogas.sum(axis=0)  # aggregate all NG (row) sectors

# assumption 4: biogas compensation takes place at the national level
# (column) recipe is preserved
new_EU27_biogas = EU27_biogas.copy()
for country in idx_EU27:
    curr_val = EU27_biogas.loc[country, country].values
    xtra_val = increase_in_biogas[country].values
    new_EU27_biogas.loc[country, country] = curr_val + xtra_val

new_EU27_biogas

region                AT                                   
sector        Paddy rice         Wheat Cereal grains nec   
region sector                                              
AT     Biogas          0  2.002269e-09      1.234557e-09  \
BE     Biogas          0  0.000000e+00      0.000000e+00   
BG     Biogas          0  0.000000e+00      0.000000e+00   
CY     Biogas          0  0.000000e+00      0.000000e+00   
CZ     Biogas          0  0.000000e+00      0.000000e+00   
DE     Biogas          0  0.000000e+00      0.000000e+00   
DK     Biogas          0  0.000000e+00      0.000000e+00   
EE     Biogas          0  0.000000e+00      0.000000e+00   
ES     Biogas          0  0.000000e+00      0.000000e+00   
FI     Biogas          0  0.000000e+00      0.000000e+00   
FR     Biogas          0  0.000000e+00      0.000000e+00   
GR     Biogas          0  0.000000e+00      0.000000e+00   
HR     Biogas          0  0.000000e+00      0.000000e+00   
HU     Biogas          0  0.000000e+00      0.000000e+00   
IE     Biogas          0  0.000000e+00      0.000000e+00   
IT     Biogas          0  0.000000e+00      0.000000e+00   
LT     Biogas          0  0.000000e+00      0.000000e+00   
LU     Biogas          0  0.000000e+00      0.000000e+00   
LV     Biogas          0  0.000000e+00      0.000000e+00   
MT     Biogas          0  0.000000e+00      0.000000e+00   
NL     Biogas          0  0.000000e+00      0.000000e+00   
PL     Biogas          0  0.000000e+00      0.000000e+00   
PT     Biogas          0  0.000000e+00      0.000000e+00   
RO     Biogas          0  0.000000e+00      0.000000e+00   
SE     Biogas          0  0.000000e+00      0.000000e+00   
SI     Biogas          0  0.000000e+00      0.000000e+00   
SK     Biogas          0  0.000000e+00      0.000000e+00   

region                                                                       
sector        Vegetables, fruit, nuts     Oil seeds Sugar cane, sugar beet   
region sector                                                                
AT     Biogas            9.247221e-10  2.168623e-09           3.456375e-09  \
BE     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
BG     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
CY     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
CZ     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
DE     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
DK     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
EE     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
ES     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
FI     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
FR     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
GR     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
HR     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
HU     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
IE     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
IT     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
LT     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
LU     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
LV     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
MT     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
NL     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
PL     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
PT     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
RO     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
SE     Biogas            0.000000e+00  0.000000e+00           0.000000e+00   
SI     Biogas  

In [20]:
# Sanity check

before_M1 = A.loc[idx_EU27_biogas.union(idx_RU_NG), idx_EU27]
after_M1 = pd.concat([new_EU27_biogas, new_RU_gas_to_EU27], axis=0)

if SWK_biogas == 1:
    flag = np.allclose(before_M1.sum(axis=0), after_M1.sum(axis=0))
    print(flag)

# Measure 2: Scale Wind and Solar

In [21]:
wind = "Electricity by wind"
solar = "Electricity by solar photovoltaic"
idx_RES = [wind, solar]
idx_EU27_RES = pd.MultiIndex.from_product(iterables=[idx_EU27, idx_RES])
EU27_RES = A.loc[idx_EU27_RES] # ! columns?


idx_CCGT = ["Electricity by gas"]
idx_EU27_CCGT = pd.MultiIndex.from_product(iterables=[idx_EU27, idx_CCGT])
RU_NG_to_EU_CCGT = A.loc[idx_RU_NG[0], idx_EU27_CCGT]  # we only keep idx_NG[0] indicator
RU_NG_to_EU_CCGT

region  sector            
AT      Electricity by gas    0.000000e+00
BE      Electricity by gas    0.000000e+00
BG      Electricity by gas    2.402647e-09
CY      Electricity by gas    0.000000e+00
CZ      Electricity by gas    7.014306e-01
DE      Electricity by gas    0.000000e+00
DK      Electricity by gas    0.000000e+00
EE      Electricity by gas    9.996377e-06
ES      Electricity by gas    0.000000e+00
FI      Electricity by gas    0.000000e+00
FR      Electricity by gas    0.000000e+00
GR      Electricity by gas    0.000000e+00
HR      Electricity by gas    0.000000e+00
HU      Electricity by gas    2.614270e-01
IE      Electricity by gas    0.000000e+00
IT      Electricity by gas    8.297285e-02
LT      Electricity by gas    5.073007e-01
LU      Electricity by gas    0.000000e+00
LV      Electricity by gas    3.626401e-05
MT      Electricity by gas    0.000000e+00
NL      Electricity by gas    0.000000e+00
PL      Electricity by gas    0.000000e+00
PT      Electricity by gas 

In [22]:
# assumption 1: reduction x % in Energy = x % in monetary flow
# assumption 2: country's reduction is proportional to its demand
# This case we only look at "Natural gas and services related to..."
# to avoid negative values and because other sectors are very low/null
import_share = RU_NG_to_EU_CCGT / RU_NG_to_EU_CCGT.sum()
RU_NG_reduction_from_RES = impact_coeff_RES * A.loc[idx_RU_NG[0], idx_EU27]
RU_NG_reduction_from_RES = import_share * RU_NG_reduction_from_RES.sum()

# Result 1
new_RU_NG_to_EU_CCGT = RU_NG_to_EU_CCGT - RU_NG_reduction_from_RES
new_RU_NG_to_EU_CCGT = new_RU_NG_to_EU_CCGT.to_frame().T  # preserve original shape
new_RU_NG_to_EU_CCGT

,region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,LU,LV,MT,NL,PL,PT,RO,SE,SI,SK
,sector,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,...,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas
RU,"Natural gas and services related to natural gas extraction, excluding surveying",0.0,0.0,-5.456091e-10,0.0,-0.159286,0.0,0.0,-0.000002,0.0,0.0,...,0.0,-0.000008,0.0,0.0,0.0,0.0,0.0,0.0,-5.276825e-07,-9.538773e-08


In [23]:
# to preserve sector's inputs recipe, we need to reallocate
# CCGT's need for gas to come from other countries
# assumption 3: reallocation comes 50% from US, 50% from Qatar
idx_USWM_LNG = pd.MultiIndex.from_product(iterables=[["US", "WM"], [idx_NG[0]]])
USWM_LNG_to_EU_CCGT = A.loc[idx_USWM_LNG, idx_EU27_CCGT]

USWM_LNG_xtra = (
        pd.concat([
                0.5*RU_NG_reduction_from_RES, 
                0.5*RU_NG_reduction_from_RES], axis=1)
        .T
        .set_index(idx_USWM_LNG)
)
# Result 2
new_USWM_LNG_to_EU_CCGT = USWM_LNG_to_EU_CCGT + USWM_LNG_xtra
new_USWM_LNG_to_EU_CCGT

,region,AT,BE,BG,CY,CZ,DE,DK,EE,ES,FI,...,LU,LV,MT,NL,PL,PT,RO,SE,SI,SK
,sector,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,...,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas,Electricity by gas
region,sector,,,,,,,,,,,,,,,,,,,,,
US,"Natural gas and services related to natural gas extraction, excluding surveying",0.0,1.941430e-09,1.474128e-09,0.0,0.430358,0.0,0.0,0.000006,1.292143e-07,0.0,...,0.0,0.000022,0.0,9.998390e-10,0.0,0.0,0.0,0.000007,0.000001,2.577189e-07
WM,"Natural gas and services related to natural gas extraction, excluding surveying",0.0,0.000000e+00,1.474128e-09,0.0,0.430358,0.0,0.0,0.000006,0.000000e+00,0.0,...,0.0,0.000022,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.000001,2.577189e-07


In [24]:
# assumption 4: Reduction of x % in CCGT output = reduction of x/0.44 % in NG requirements
# assuming a 44% energy efficiency and assuming that
# what holds for energy flows, holds for monetary flows
coeff_CCGT = technical_coeff_CCGT * import_share
EU_CCGT = A.loc[idx_EU27_CCGT]
EU_CCGT_reduction_from_RES = EU_CCGT.multiply(coeff_CCGT, axis=0)

# Result 3
new_EU_CCGT = EU_CCGT - EU_CCGT_reduction_from_RES
new_EU_CCGT

region                            AT                                   
sector                    Paddy rice         Wheat Cereal grains nec   
region sector                                                          
AT     Electricity by gas        0.0  4.942024e-04      3.901775e-04  \
BE     Electricity by gas        0.0  1.241026e-07      9.798020e-08   
BG     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
CY     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
CZ     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
DE     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
DK     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
EE     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
ES     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
FI     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
FR     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
GR     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
HR     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
HU     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
IE     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
IT     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
LT     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
LU     Electricity by gas        0.0  6.260886e-07      4.943030e-07   
LV     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
MT     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
NL     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
PL     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
PT     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
RO     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
SE     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
SI     Electricity by gas        0.0  0.000000e+00      0.000000e+00   
SK     Electricity by gas        0.0  0.000000e+00      0.000000e+00   

region                                                            
sector                    Vegetables, fruit, nuts     Oil seeds   
region sector                                                     
AT     Electricity by gas            2.292064e-03  1.722978e-04  \
BE     Electricity by gas            5.755761e-07  4.326691e-08   
BG     Electricity by gas            0.000000e+00  0.000000e+00   
CY     Electricity by gas            0.000000e+00  0.000000e+00   
CZ     Electricity by gas            0.000000e+00  0.000000e+00   
DE     Electricity by gas            0.000000e+00  0.000000e+00   
DK     Electricity by gas            0.000000e+00  0.000000e+00   
EE     Electricity by gas            0.000000e+00  0.000000e+00   
ES     Electricity by gas            0.000000e+00  0.000000e+00   
FI     Electricity by gas            0.000000e+00  0.000000e+00   
FR     Electricity by gas            0.000000e+00  0.000000e+00   
GR     Electricity by gas            0.000000e+00  0.000000e+00   
HR     Electricity by gas            0.000000e+00  0.000000e+00   
HU     Electricity by gas            0.000000e+00  0.000000e+00   
IE     Electricity by gas            0.000000e+00  0.000000e+00   
IT     Electricity by gas            0.000000e+00  0.000000e+00   
LT     Electricity by gas            0.000000e+00  0.000000e+00   
LU     Electricity by gas            2.903740e-06  2.182784e-07   
LV     Electricity by gas            0.000000e+00  0.000000e+00   
MT     Electricity by gas            0.000000e+00  0.000000e+00   
NL     Electricity by gas            0.000000e+00  0.000000e+00   
PL     Electricity by gas            0.000000e+00  0.000000e+00   
PT     Electricity by gas            0.000000e+00  0.000000e+00   
RO     Electricity by gas            0.000000e+00  0.000000e+00   
SE     Electricity by gas     

In [25]:
# Assumption 5: apply LCOE data to compute price coeff
# using averages for european countries with default settings
# data source: https://www.iea.org/data-and-statistics/data-tools/levelised-cost-of-electricity-calculator
price_coeff_wind = LCOE_gas / LCOE_wind
technical_coeff_wind = 1
SWK_wind = price_coeff_wind * technical_coeff_wind
increase_in_wind = EU_CCGT_reduction_from_RES / SWK_wind # if 100% substitution by wind
increase_in_wind = increase_in_wind.rename(index={idx_CCGT[0]: wind})

price_coeff_solar = LCOE_gas / LCOE_solar
technical_coeff_solar = 1
SWK_solar = price_coeff_solar * technical_coeff_solar
increase_in_solar = EU_CCGT_reduction_from_RES / SWK_solar  # if 100% substitution by solar
increase_in_solar = increase_in_solar.rename(index={idx_CCGT[0]: solar})

increase_in_solar

region                                           AT                           
sector                                   Paddy rice Wheat Cereal grains nec   
region sector                                                                 
AT     Electricity by solar photovoltaic        0.0   0.0               0.0  \
BE     Electricity by solar photovoltaic        0.0   0.0               0.0   
BG     Electricity by solar photovoltaic        0.0   0.0               0.0   
CY     Electricity by solar photovoltaic        0.0   0.0               0.0   
CZ     Electricity by solar photovoltaic        0.0   0.0               0.0   
DE     Electricity by solar photovoltaic        0.0   0.0               0.0   
DK     Electricity by solar photovoltaic        0.0   0.0               0.0   
EE     Electricity by solar photovoltaic        0.0   0.0               0.0   
ES     Electricity by solar photovoltaic        0.0   0.0               0.0   
FI     Electricity by solar photovoltaic        0.0   0.0               0.0   
FR     Electricity by solar photovoltaic        0.0   0.0               0.0   
GR     Electricity by solar photovoltaic        0.0   0.0               0.0   
HR     Electricity by solar photovoltaic        0.0   0.0               0.0   
HU     Electricity by solar photovoltaic        0.0   0.0               0.0   
IE     Electricity by solar photovoltaic        0.0   0.0               0.0   
IT     Electricity by solar photovoltaic        0.0   0.0               0.0   
LT     Electricity by solar photovoltaic        0.0   0.0               0.0   
LU     Electricity by solar photovoltaic        0.0   0.0               0.0   
LV     Electricity by solar photovoltaic        0.0   0.0               0.0   
MT     Electricity by solar photovoltaic        0.0   0.0               0.0   
NL     Electricity by solar photovoltaic        0.0   0.0               0.0   
PL     Electricity by solar photovoltaic        0.0   0.0               0.0   
PT     Electricity by solar photovoltaic        0.0   0.0               0.0   
RO     Electricity by solar photovoltaic        0.0   0.0               0.0   
SE     Electricity by solar photovoltaic        0.0   0.0               0.0   
SI     Electricity by solar photovoltaic        0.0   0.0               0.0   
SK     Electricity by solar photovoltaic        0.0   0.0               0.0   

region                                                                       
sector                                   Vegetables, fruit, nuts Oil seeds   
region sector                                                                
AT     Electricity by solar photovoltaic                     0.0       0.0  \
BE     Electricity by solar photovoltaic                     0.0       0.0   
BG     Electricity by solar photovoltaic                     0.0       0.0   
CY     Electricity by solar photovoltaic                     0.0       0.0   
CZ     Electricity by solar photovoltaic                     0.0       0.0   
DE     Electricity by solar photovoltaic                     0.0       0.0   
DK     Electricity by solar photovoltaic                     0.0       0.0   
EE     Electricity by solar photovoltaic                     0.0       0.0   
ES     Electricity by solar photovoltaic                     0.0       0.0   
FI     Electricity by solar photovoltaic                     0.0       0.0   
FR     Electricity by solar photovoltaic                     0.0       0.0   
GR     Electricity by solar photovoltaic                     0.0       0.0   
HR     Electricity by solar photovoltaic                     0.0       0.0   
HU     Electricity by solar photovoltaic                     0.0       0.0   
IE     Electricity by solar photovoltaic                     0.0       0.0   
IT     Electricity by solar photovoltaic                     0.0       0.0   
LT     Electricity by solar photovoltaic                     0.0       0.0   
LU     Electricity by solar photovoltaic                     0.0     

In [26]:
# assumption 6: electricity mix compensation takes place at the national level
# and increase proportionally to current Solar vs Wind production of the country
country_mapping = pd.read_excel(annex_file, sheet_name="Country_Code", header=0, index_col=0)
country_mapping = country_mapping.to_dict()['Code']

share_SW = pd.read_excel(annex_file, sheet_name="Solar_Wind", index_col=0)
share_SW = share_SW.rename(index=country_mapping).loc[idx_EU27][["Solar (%)", "Wind (%)"]]

# slow because not vectorized (not enough time)
new_EU27_RES = EU27_RES.copy()
for country in idx_EU27:
    curr_wind = EU27_RES.loc[(country, wind)]
    xtra_wind = increase_in_wind.loc[(country, wind)]
    share_wind = share_SW.loc[country, "Wind (%)"]
    new_wind = curr_wind + share_wind * xtra_wind
    new_EU27_RES.loc[(country, wind)] = new_wind.values

    curr_solar = EU27_RES.loc[(country, solar)]
    xtra_solar = increase_in_solar.loc[(country, solar)]
    share_solar = share_SW.loc[country, "Solar (%)"]
    new_solar = curr_solar + share_solar * xtra_solar
    new_EU27_RES.loc[(country, solar)] = new_solar.values

# Result 4
new_EU27_RES

region                                           AT                 
sector                                   Paddy rice         Wheat   
region sector                                                       
AT     Electricity by wind                        0  7.870654e-05  \
       Electricity by solar photovoltaic          0  7.270232e-06   
BE     Electricity by wind                        0  5.177819e-09   
       Electricity by solar photovoltaic          0  1.607425e-09   
BG     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
CY     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
CZ     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
DE     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
DK     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
EE     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
ES     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
FI     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
FR     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
GR     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
HR     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
HU     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
IE     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
IT     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
LT     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
LU     Electricity by wind                        0  1.495972e-08   
       Electricity by solar photovoltaic          0  1.107962e-08   
LV     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
MT     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
NL     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
PL     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
PT     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
RO     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
SE     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
SI     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   
SK     Electricity by wind                        0  0.000000e+00   
       Electricity by solar photovoltaic          0  0.000000e+00   

region                                                       
sect

In [27]:
# Here are the intermediary results of Measure 2

measures = (
    new_RU_NG_to_EU_CCGT,  # Result 1
    new_USWM_LNG_to_EU_CCGT,  # Result 2
    new_EU_CCGT,  # Result 3
    new_EU27_RES,  # Result 4
)

idx = (
    new_RU_NG_to_EU_CCGT.index
    .union(new_USWM_LNG_to_EU_CCGT.index)
    .union(new_EU_CCGT.index)
    .union(new_EU27_RES.index)
)

before_M2 = A.loc[idx]
after_M2 = before_M2.copy()

for df_ in measures:
    after_M2.loc[df_.index, df_.columns] = df_.values
    

# Sanity check
if (SWK_wind == 1) & (SWK_solar == 1) & (technical_coeff_CCGT == 1):
    flag = np.allclose(
        before_M2.groupby(level=0, axis=1).sum().sum(), 
        after_M2.groupby(level=0, axis=1).sum().sum()
    )
    print(flag)

# Results

In [28]:
A1 = A.copy()
A1.loc[after_M1.index, after_M1.columns] = after_M1.values

A2 = A.copy()
A2.loc[after_M2.index, after_M2.columns] = after_M2.values

# # Save files (very slow, uncomment only if needed)
# A1.to_csv('../data/A_2019_pxp_biogas.csv')
# A2.to_csv('../data/A_2019_pxp_elec_mix.csv')